In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark =SparkSession.builder \
  .master('local[*]') \
  .appName('conversions') \
  .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.23.2') \
  .getOrCreate() 
sqlContext = SQLContext(spark)

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bf0b2c4c-6d0b-492d-ae22-03a1dc282074;1.0
	confs: [default]
	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.23.2 in central
:: resolution report :: resolve 335ms :: artifacts dl 6ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.23.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrie

In [2]:
df = sqlContext.read.format("csv").option("header","true").option("delimiter", ",").load("gs://dataproc-staging-asia-southeast1-486650843789-dpb1jzlc/NFT_Sales.csv")
df.show(5)

+----------+----------------+----------------------+----------------------------+--------------------+----------------------+--------------+-----------------+---------------+---------------------+-------------+
|      Date|Sales_USD_cumsum|Number_of_Sales_cumsum|Active_Market_Wallets_cumsum|Primary_Sales_cumsum|Secondary_Sales_cumsum|AverageUSD_cum|        Sales_USD|Number_of_Sales|Active_Market_Wallets|Primary_Sales|
+----------+----------------+----------------------+----------------------------+--------------------+----------------------+--------------+-----------------+---------------+---------------------+-------------+
|2017-06-22|             0.0|                     0|                         0.0|                   0|                  null|          null|             null|           null|                 null|         null|
|2017-06-23|          1020.3|                    19|                         8.0|                   0|                  19.0|          53.7|           1020.

In [3]:
df.count()

1606

In [4]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Sales_USD_cumsum: string (nullable = true)
 |-- Number_of_Sales_cumsum: string (nullable = true)
 |-- Active_Market_Wallets_cumsum: string (nullable = true)
 |-- Primary_Sales_cumsum: string (nullable = true)
 |-- Secondary_Sales_cumsum: string (nullable = true)
 |-- AverageUSD_cum: string (nullable = true)
 |-- Sales_USD: string (nullable = true)
 |-- Number_of_Sales: string (nullable = true)
 |-- Active_Market_Wallets: string (nullable = true)
 |-- Primary_Sales: string (nullable = true)



Find NA values in this table. Only Owners are Na, so changing the value to -1
-----------------------------------------

In [5]:
df.filter(df.Sales_USD.isNull()).show()

+----------+----------------+----------------------+----------------------------+--------------------+----------------------+--------------+---------+---------------+---------------------+-------------+
|      Date|Sales_USD_cumsum|Number_of_Sales_cumsum|Active_Market_Wallets_cumsum|Primary_Sales_cumsum|Secondary_Sales_cumsum|AverageUSD_cum|Sales_USD|Number_of_Sales|Active_Market_Wallets|Primary_Sales|
+----------+----------------+----------------------+----------------------------+--------------------+----------------------+--------------+---------+---------------+---------------------+-------------+
|2017-06-22|             0.0|                     0|                         0.0|                   0|                  null|          null|     null|           null|                 null|         null|
+----------+----------------+----------------------+----------------------------+--------------------+----------------------+--------------+---------+---------------+---------------------+

In [6]:
df.show()

+----------+----------------+----------------------+----------------------------+--------------------+----------------------+--------------+------------------+---------------+---------------------+-------------+
|      Date|Sales_USD_cumsum|Number_of_Sales_cumsum|Active_Market_Wallets_cumsum|Primary_Sales_cumsum|Secondary_Sales_cumsum|AverageUSD_cum|         Sales_USD|Number_of_Sales|Active_Market_Wallets|Primary_Sales|
+----------+----------------+----------------------+----------------------------+--------------------+----------------------+--------------+------------------+---------------+---------------------+-------------+
|2017-06-22|             0.0|                     0|                         0.0|                   0|                  null|          null|              null|           null|                 null|         null|
|2017-06-23|          1020.3|                    19|                         8.0|                   0|                  19.0|          53.7|            

In [7]:
df = df.withColumn("Sales_USD", df["Sales_USD"].cast(DoubleType()))
df = df.na.fill(0)
df.show(5)
df.printSchema()

+----------+----------------+----------------------+----------------------------+--------------------+----------------------+--------------+-----------------+---------------+---------------------+-------------+
|      Date|Sales_USD_cumsum|Number_of_Sales_cumsum|Active_Market_Wallets_cumsum|Primary_Sales_cumsum|Secondary_Sales_cumsum|AverageUSD_cum|        Sales_USD|Number_of_Sales|Active_Market_Wallets|Primary_Sales|
+----------+----------------+----------------------+----------------------------+--------------------+----------------------+--------------+-----------------+---------------+---------------------+-------------+
|2017-06-22|             0.0|                     0|                         0.0|                   0|                  null|          null|              0.0|           null|                 null|         null|
|2017-06-23|          1020.3|                    19|                         8.0|                   0|                  19.0|          53.7|           1020.

In [8]:
df.write.format('bigquery').option("temporaryGcsBucket","temp_bucket_temp").option('table', 'nftSales.nft-Sales').mode('overwrite').save()

In [9]:
#df.write.format('bigquery').option("temporaryGcsBucket","temp_buc_ket").option('table', 'Sample_run.historySales_cropped').mode('overwrite').save()